In [1]:
!pip install h2o

     |████████████████████████████████| 177.0 MB 12 kB/s 
  Created wheel for h2o: filename=h2o-3.36.1.1-py2.py3-none-any.whl size=177068062 sha256=d26e3e1aac3f50708f4b0c71e180889686e9b81a3df79be360e291fddf4d5f9d
  Stored in directory: /root/.cache/pip/wheels/a6/d9/ab/5442447c7e2ccf07f66aa8b79f3877ce5382f0b95e6c0c797b
Successfully built h2o


In [2]:
import h2o
from h2o.automl import H2OAutoML
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import sklearn

In [3]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.15" 2022-04-19; OpenJDK Runtime Environment (build 11.0.15+10-Ubuntu-0ubuntu0.18.04.1); OpenJDK 64-Bit Server VM (build 11.0.15+10-Ubuntu-0ubuntu0.18.04.1, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpnnyfq7vv
  JVM stdout: /tmp/tmpnnyfq7vv/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpnnyfq7vv/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,05 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.1
H2O_cluster_version_age:,18 days
H2O_cluster_name:,H2O_from_python_unknownUser_a0w5f9
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.172 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [4]:
import pandas as pd
def dataSetup(name):
  X,y = sklearn.datasets.fetch_openml(name, as_frame=True, return_X_y=True)
  train = pd.concat([X, y], axis=1, join='inner')
  train.dropna()
  train = train.apply(lambda x: pd.factorize(x)[0])
  X,y = train.iloc[:,:-1], train.iloc[:, -1]
  # X = X.apply(lambda x: pd.factorize(x)[0])
  X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y,random_state=42)
  y_train = y_train.to_frame(name="class")
  y_test = y_test.to_frame(name="class")
  test = pd.concat([X_test, y_test], axis=1, join='inner')
  train = pd.concat([X_train, y_train], axis=1, join='inner')
  test = h2o.H2OFrame(test)
  train = h2o.H2OFrame(train)
  return (test, train)

In [5]:
test, train = dataSetup('airlines')

/usr/local/lib/python3.7/dist-packages/sklearn/datasets/_openml.py:421: UserWarning: Multiple active versions of the dataset matching the name airlines exist. Versions may be fundamentally different, returning version 1.
  " {version}.".format(name=name, version=res[0]["version"])


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [6]:
train.shape

(404537, 8)

In [7]:
x = train.columns
y = "class"
x.remove(y)

In [8]:
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [9]:
aml = H2OAutoML(max_runtime_secs = 3600)
aml.train(x=x, y=y, training_frame=train)
# train = h2o.H2OFrame(train)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_2_AutoML_1_20220501_183539

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.18992609929600757
RMSE: 0.4358051161884261
LogLoss: 0.5610193643899458
Null degrees of freedom: 9960
Residual degrees of freedom: 9953
Null deviance: 13698.771708530405
Residual deviance: 11176.6277773765
AIC: 11192.6277773765
AUC: 0.7850672617479167
AUCPR: 0.8090103269429002
Gini: 0.5701345234958335

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.45064623160753475: 


,,0,1,Error,Rate
0,0,2075.0,2382.0,0.5344,(2382.0/4457.0)
1,1,610.0,4894.0,0.1108,(610.0/5504.0)
2,Total,2685.0,7276.0,0.3004,(2992.0/9961.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.450646,0.765884,249.0
1,max f2,0.195178,0.871219,352.0
2,max f0point5,0.612346,0.748892,154.0
3,max accuracy,0.539317,0.719506,199.0
4,max precision,0.948580,1.000000,0.0
5,max recall,0.101077,1.000000,387.0
6,max specificity,0.948580,1.000000,0.0
7,max absolute_mcc,0.549962,0.430012,193.0
8,max min_per_class_accuracy,0.568040,0.711465,181.0
9,max mean_per_class_accuracy,0.549962,0.714616,193.0



Gains/Lift Table: Avg response rate: 55.26 %, avg score: 55.31 %


...


Gains/Lift Table: Avg response rate: 55.45 %, avg score: 55.45 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.010002,0.892951,1.653317,1.653317,0.916708,0.911275,0.916708,0.911275,0.016536,0.016536,65.331672,65.331672,0.014666
1,2,0.020001,0.872021,1.598438,1.625881,0.886279,0.881748,0.901495,0.896514,0.015983,0.032519,59.843779,62.588065,0.028097
2,3,0.030002,0.857208,1.566394,1.606050,0.868512,0.864298,0.890500,0.885774,0.015666,0.048185,56.639390,60.605010,0.040811
3,4,0.040001,0.845049,1.520857,1.584754,0.843263,0.850844,0.878692,0.877043,0.015207,0.063392,52.085671,58.475438,0.052501
4,5,0.050000,0.834174,1.489200,1.565645,0.825711,0.839388,0.868097,0.869512,0.014891,0.078283,48.920006,56.564541,0.063480
5,6,0.100001,0.791358,1.463284,1.514465,0.811341,0.811799,0.839719,0.840656,0.073165,0.151448,46.328417,51.446479,0.115472
6,7,0.150001,0.757142,1.397391,1.475440,0.774806,0.773673,0.818081,0.818328,0.069870,0.221318,39.739135,47.544031,0.160070
7,8,0.200001,0.727288,1.315984,1.435576,0.729668,0.742023,0.795978,0.799252,0.065800,0.287117,31.598386,43.557620,0.195531
8,9,0.300000,0.674522,1.267822,1.379659,0.702964,0.700221,0.764974,0.766242,0.126780,0.413897,26.782160,37.965892,0.255643
9,10,0.400000,0.626551,1.184287,1.330816,0.656647,0.650405,0.737892,0.737283,0.118430,0.532327,18.428739,33.081574,0.297006




Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.652401,0.003181,0.647869,0.654575,0.652141,0.651288,0.656131
1,auc,0.723095,0.000668,0.723130,0.723510,0.722713,0.722206,0.723915
2,err,0.347599,0.003181,0.352131,0.345425,0.347859,0.348712,0.343869
3,err_count,28123.200000,242.478240,28467.000000,27873.000000,28130.000000,28230.000000,27916.000000
4,f0point5,0.669889,0.002877,0.665259,0.671733,0.670939,0.669076,0.672439
5,f1,0.745643,0.001883,0.744271,0.745357,0.748781,0.745699,0.744106
6,f2,0.840751,0.005739,0.844581,0.837106,0.847056,0.842140,0.832872
7,lift_top_group,1.652538,0.023287,1.676627,1.674251,1.626826,1.631210,1.653779
8,logloss,0.604840,0.000578,0.605217,0.604536,0.604169,0.605625,0.604651
9,max_per_class_error,0.679346,0.020487,0.697522,0.665573,0.697129,0.685523,0.650984



See the whole table with table.as_data_frame()


In [10]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_2_AutoML_1_20220501_183539,0.723087,0.60484,0.746247,0.385151,0.457066,0.208909
StackedEnsemble_AllModels_3_AutoML_1_20220501_183539,0.723076,0.604844,0.746222,0.384677,0.457067,0.20891
StackedEnsemble_AllModels_1_AutoML_1_20220501_183539,0.723014,0.604877,0.746157,0.385748,0.457083,0.208925
StackedEnsemble_BestOfFamily_3_AutoML_1_20220501_183539,0.72223,0.605515,0.745521,0.381224,0.457382,0.209198
StackedEnsemble_BestOfFamily_2_AutoML_1_20220501_183539,0.722062,0.605623,0.74539,0.38137,0.457437,0.209249
StackedEnsemble_BestOfFamily_1_AutoML_1_20220501_183539,0.722007,0.605629,0.745216,0.386605,0.457438,0.20925
GBM_1_AutoML_1_20220501_183539,0.720861,0.606489,0.744396,0.386211,0.457855,0.209631
GBM_4_AutoML_1_20220501_183539,0.716618,0.609585,0.739754,0.384812,0.459315,0.210971
XGBoost_2_AutoML_1_20220501_183539,0.713054,0.612539,0.735642,0.387036,0.460613,0.212164
GBM_3_AutoML_1_20220501_183539,0.712342,0.612596,0.736228,0.38945,0.460751,0.212292
